In [58]:
import json
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import requests

options = webdriver.ChromeOptions()
options.set_capability('goog:loggingPrefs', {"performance": "ALL", "browser": "ALL"})
# Adding user agent to look more like a real browser
options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.36")

driver = webdriver.Chrome(options=options)
driver.set_page_load_timeout(30)

In [4]:
url = "https://www.sofascore.com/football/match/manchester-united-leicester-city/GK#id:12437027,tab:statistics"

In [47]:
def get_score(match_id, driver):
    # Clear existing logs
    #driver.get("about:blank")
    #logs = driver.get_log("performance")
    
    # Load the page
    """
    print(f"Loading URL: {url}")
    driver.get(url)
    """
    
    """
    # Wait for the page to load properly
    try:
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "body"))
        )
    except:
        print("Timeout waiting for page to load")
    """
    """
    # Interact with the page to trigger API calls
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight/2);")
    time.sleep(0.1)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(0.2)

    soup = BeautifulSoup (driver.page_source, "html.parser")

    meta_data = soup.find_all('script', type = 'application/ld+json')

    meta_data = json.loads(meta_data[1].string)
    """
    # Direct API approach - try to call the API directly
    
    if match_id:
        print(f"Extracted match ID: {match_id}")
        urls = []
        urls.append(f"https://www.sofascore.com/api/v1/event/{match_id}/statistics")
        urls.append(f"https://www.sofascore.com/api/v1/event/{match_id}")
        urls.append(f"https://www.sofascore.com/api/v1/event/{match_id}/lineups")

        
        data = {}
        for i in range(0,len(urls)):
            api_url = urls[i]
            # Open the API URL directly
            #driver.execute_script(f"window.open('{api_url}', '_blank');")

            driver.get(api_url)
            #time.sleep(2)
            
            # Switch to the new tab
            #driver.switch_to.window(driver.window_handles[-1])
            #time.sleep(1)
            
            # Get the page source which should contain the JSON
            page_source = driver.page_source
            
            # Extract JSON from the page source
            if "application/json" in page_source or "{" in page_source:
                start_idx = page_source.find("{")
                end_idx = page_source.rfind("}") + 1
                if start_idx >= 0 and end_idx > start_idx:
                    json_str = page_source[start_idx:end_idx]
                    try:
                        data[i] = json.loads(json_str)
                    except json.JSONDecodeError:
                        print("Failed to parse JSON response")
            
            # Close the tab and switch back
            #driver.close()
            #driver.switch_to.window(driver.window_handles[0])

        #data[len(urls)] = meta_data
        return data

In [48]:
data =  (get_score(12437027,driver))

Extracted match ID: 12437027


In [53]:
def parse_data(data,cat):
    ## statisitics = 0 
    ## event_data = 1
    ## formation = 2
    this = cat
    opp = "away" if cat == "home" else "home"
    
    statistics = data[0]['statistics'][0]['groups']
    event = data[1]['event']
    lineups = data[2]
    
    match_data = {}

    """
    event_header = ['ManagerName','TeamName','Score']
    event_header_info = ['roundInfo','winnerCode']
    """
    
    match_data['home'] = "1" if this == "home" else "0"
    
    match_data['thisTeamName'] = event[this + 'Team']['slug']
    match_data['oppTeamName'] = event[opp + 'Team']['slug']
    match_data['thisManagerName'] = event[this + 'Team']['manager']['slug']
    match_data['oppManagerName'] = event[opp + 'Team']['manager']['slug']
    match_data['thisScore'] = str(event[this + 'Score']['current'])
    match_data['oppScore'] = str(event[opp + 'Score']['current'])

    match_data['roundInfo'] = str(event['roundInfo']['round'])
    match_data['winnerCode'] = str(event['winnerCode'])
    
    ## get stats data from match 
    stats_headers = ['Ball possession','Expected goals','Big chances', 'Total shots', 'Goalkeeper saves','Corner kicks','Fouls','Passes','Tackles','Free kicks','Yellow cards','Red cards']
    stats_shots_headers = ['Total shots', 'Shots on target','Hit woodwork','Shots off target','Blocked shots', 'Shots inside box','Shots outside box']
    stats_attack_headers = ['Big chances missed','Through balls','Touches in penalty area','Fouled in final third', 'Offsides']
    stats_passes_headers = ['Accurate passes','Throw-ins','Final third entries','Long balls', 'crosses']
    stats_duels_headers = ['Duels','Dispossessed','Ground duels','Aerial duels','Dribbles']
    stats_defending_headers = ['Total tackles', 'Interceptions','Recoveries','Clearances','Errors lead to a shot']
    stats_goalkeeper_headers = ['Total saves','Goals prevented','Big saves','High claims','Goal kicks']

    all_headers = (
        stats_headers +
        stats_shots_headers +
        stats_attack_headers +
        stats_passes_headers +
        stats_duels_headers +
        stats_defending_headers +
        stats_goalkeeper_headers
    )
    match_data.update({'this' + header: "0" for header in all_headers})
    match_data.update({'opp' + header: "0" for header in all_headers})
    
    # parse stats API 
    for group in statistics:
        for stat in group['statisticsItems']:
            if stat['name'] in all_headers:
                match_data['this' + stat['name']] = str(stat[this + 'Value'])
                match_data['opp' + stat['name']] = str(stat[opp + 'Value'])
    return (match_data)

In [49]:
print(parse_data(data,'home'))
print (parse_data(data,'away'))

{'home': '1', 'thisTeamName': 'leicester-city', 'oppTeamName': 'manchester-united', 'thisManagerName': 'ruud-van-nistelrooy', 'oppManagerName': 'ruben-amorim', 'thisScore': 0, 'oppScore': 3, 'roundInfo': 29, 'winnerCode': 2, 'thisBall possession': 54, 'thisExpected goals': 1.05, 'thisBig chances': 3, 'thisTotal shots': 11, 'thisGoalkeeper saves': 2, 'thisCorner kicks': 6, 'thisFouls': 6, 'thisPasses': 520, 'thisTackles': 21, 'thisFree kicks': 6, 'thisYellow cards': 2, 'thisRed cards': '0', 'thisShots on target': 3, 'thisHit woodwork': 0, 'thisShots off target': 2, 'thisBlocked shots': 6, 'thisShots inside box': 7, 'thisShots outside box': 4, 'thisBig chances missed': 3, 'thisThrough balls': 1, 'thisTouches in penalty area': 23, 'thisFouled in final third': 1, 'thisOffsides': 4, 'thisAccurate passes': 434, 'thisThrow-ins': 16, 'thisFinal third entries': 49, 'thisLong balls': 18, 'thiscrosses': '0', 'thisDuels': 48, 'thisDispossessed': 12, 'thisGround duels': 34, 'thisAerial duels': 17, 

In [59]:
file = open("match_links_23_24.txt", "r")
for match_id in file:
    match_id = match_id.strip()
    data = get_score(match_id,driver)
    home_data = parse_data(data,'home')
    away_data = parse_data(data,'away')
    
    with open("match_23_24.csv", "a") as f:
        f.write (",".join(home_data.values()) + "\n")
        f.write (",".join(away_data.values()) + "\n")

f.close()

Extracted match ID: 11352326
Extracted match ID: 11352331
Extracted match ID: 11352481
Extracted match ID: 11352560
Extracted match ID: 11352469
Extracted match ID: 11352381
Extracted match ID: 11352609
Extracted match ID: 12240634
Extracted match ID: 11352397
Extracted match ID: 11352419
Extracted match ID: 11352476
Extracted match ID: 11352551
Extracted match ID: 11352327
Extracted match ID: 11352440
Extracted match ID: 11352546
Extracted match ID: 11352334
Extracted match ID: 11352350
Extracted match ID: 11352618
Extracted match ID: 11352508
Extracted match ID: 11352599
Extracted match ID: 11352345
Extracted match ID: 11352498
Extracted match ID: 11352443
Extracted match ID: 11352541
Extracted match ID: 11352365
Extracted match ID: 11352651
Extracted match ID: 11352342
Extracted match ID: 11352323
Extracted match ID: 11352373
Extracted match ID: 11352353
Extracted match ID: 11352600
Extracted match ID: 11352325
Extracted match ID: 11352501
Extracted match ID: 11352527
Extracted matc